## Set Up Environment

In [ ]:
# Cell 0

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20210124194550.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://52.53.175.151:9000",       "nid": 3}}

env = connections[network]

In [ ]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
# Cell 3

wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)

### Send ICX to Balanced test wallet

In [ ]:
# Cell 4
# Only necessary if running locally or on the private tbears server.

test_wallet = KeyWallet.load("./keystores/keystore_test1.json", "test1_Account")
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, test_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

### Deployed to testnet

In [ ]:
# Cell 5
# The following addresses are those deployed to the testnet at the start of the audit.

# Uncomment these lines and comment out the definition below to load contracts from a saved file.
# with open(contracts_file_custom, 'rb') as f:
#     contracts = pkl.load(f)

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxcd9ae763fabcf6c3c9a475f727264e9cfffdfaf2'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx52aab6b1dd316396e684573464af36c634059721'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx02bba98bf78c91329c6958c61ceed1e7bad3b68a'},
 'reserve': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cx418f4a6392b88568cb9587c6ca638b358f8ca728'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx65fb9d6c9bafdc9cdbe6bf258fd31b90f2f4ab8d'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx291dacbb875a94b364194a5febaac4e6318681f7'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx4179247372b8ce4c9edd8013dd3918852bd0c7fa'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx5b0d6a5b244e9254ca05592389672c5185d9f5f8'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cx699694fa8dbb76d904d33b63f90dee7cf0440e71'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxd89295b894b72b3b5966848f6fe2bbb997f99e71'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx6fda38cfc5c828763306ad7fb4225a8062bd0aee'}}

In [ ]:
# Private tbears server - with 1 hour day.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxff5a3bd8427cbb7e0683a580b0a64ea6797fe4bc'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxe2a77d212a52eb4c4168050fb806c8d722ddedb1'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx38d297206ab5d4b4b3c76ac185cdcb67f30f5b36'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx327e7a0324dc7faff3924e518165976df00b77e0'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx875e82b04c649e2884bcea18a05c8692d3555294'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx4cecbb5e373fa48180ac00d824ffa750f9f775b6'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxc60f7b9c24791d755be8e1198946807e243c24fb'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx0478fad07631c13f58b5527e05397845cb3fb1e9'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxc4d4e7b604809c922c80adb39148bbe4e0af6d64'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxa8c24edf6af229c849e9c61655dd3e867c85f4e4'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxc2b6b7cf8a9703688a01eced8a15418b07395ea8'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx443723e570db1fc8b493ab22b4520656ca3ee854'}}

In [2068]:
# Cell 6

deploy = list(contracts.keys())[:]
deploy.remove('dummy_oracle')
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

contracts

{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx21ecd939487863d4528d735543342d8e87cbfe01'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxb428669efeffd4e22d5d898c80609770c69069e7'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx50d45db0afba3719e1aab9de8e46a2391ae1a77e'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx5e11e2165adde4e132167bb98edc36a95f77e9ae'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx21ac0b34d50d853d47e071fa6b3b4fd0d156b300'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx6eb8cc8c2ddc12021e8cb48d30aeb6afa8524aba'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx0478fad07631c13f58b5527e05397845cb3fb1e9'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19'},
 'icd': {'zip': 'token_co

## Deploy All SCOREs

In [2037]:
# Cell 7
# prints out contracts dictionary with all new contract addresses

step_limit = 4000100000

for score in deploy:
    loc = contracts[score]
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

#     step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
    print(score)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('\n')

contracts

Retrying in 1 seconds...
Retrying in 2 seconds...
loans
Status: 1


Retrying in 1 seconds...
staking
Status: 1


Retrying in 1 seconds...
Retrying in 2 seconds...
dividends
Status: 1


Retrying in 1 seconds...
reserve
Status: 1


Retrying in 1 seconds...
Retrying in 2 seconds...
rewards
Status: 1


Retrying in 1 seconds...
Retrying in 2 seconds...
dex
Status: 1


Retrying in 1 seconds...
governance
Status: 1


Retrying in 1 seconds...
Retrying in 2 seconds...
sicx
Status: 1


Retrying in 1 seconds...
icd
Status: 1


Retrying in 1 seconds...
Retrying in 2 seconds...
bal
Status: 1


Retrying in 1 seconds...
bwt
Status: 1




{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx21ecd939487863d4528d735543342d8e87cbfe01'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxb428669efeffd4e22d5d898c80609770c69069e7'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx50d45db0afba3719e1aab9de8e46a2391ae1a77e'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx5e11e2165adde4e132167bb98edc36a95f77e9ae'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx21ac0b34d50d853d47e071fa6b3b4fd0d156b300'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx6eb8cc8c2ddc12021e8cb48d30aeb6afa8524aba'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx0478fad07631c13f58b5527e05397845cb3fb1e9'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19'},
 'icd': {'zip': 'token_co

In [ ]:
# Cell 8
# saves contracts dictionary to file

# now = datetime.utcnow()
# filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
# print(f'saving contracts list as {filename}')
# with open(filename, 'wb') as f:
#     pkl.dump(contracts, f)

In [2049]:
# Cell 9
# Prints out dictionary of SCORE addresses for use in testing UI

def get_scores_json(contracts):
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

print(get_scores_json(contracts))

{"loans": "cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407", "staking": "cx21ecd939487863d4528d735543342d8e87cbfe01", "dividends": "cxb428669efeffd4e22d5d898c80609770c69069e7", "reserve": "cx50d45db0afba3719e1aab9de8e46a2391ae1a77e", "rewards": "cx5e11e2165adde4e132167bb98edc36a95f77e9ae", "dex": "cx21ac0b34d50d853d47e071fa6b3b4fd0d156b300", "governance": "cx6eb8cc8c2ddc12021e8cb48d30aeb6afa8524aba", "dummy_oracle": "cx0478fad07631c13f58b5527e05397845cb3fb1e9", "sicx": "cxcfc39861330419a80f0d9f37a4cc8b7d80605a19", "icd": "cx249d6262fdc03fdc791a556ef92d8560f9818b77", "bal": "cxba676d21fe0637cf249093fe4fa8f17c90822e2d", "bwt": "cx2c4bf4c30c6e7a1c41fb3af9940cec11d98ddf1f"}


### Deploy a single SCORE

In [ ]:
# Cell 10

# Deploy a single SCORE
contract = 'bal'
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

step_limit = 10000000000


signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
contracts[contract]["SCORE"] = res.get('scoreAddress', '')
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

## Update a SCORE

In [2069]:
# Cell 11

contract = 'loans'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

# step_limit = icon_service.estimate_step(deploy_transaction) + 100000
step_limit = 3000000000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0xad78ebc5ac6200000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'indexed': ['Mint(Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0xad78ebc5ac6200000', '0x4e6f6e65']} 



## Set parameters

In [2038]:
# Cell 12

from typing import List, TypedDict

addresses = {'loans': contracts['loans']['SCORE'],
             'dex': contracts['dex']['SCORE'],
             'staking': contracts['staking']['SCORE'],
             'rewards': contracts['rewards']['SCORE'],
             'reserve': contracts['reserve']['SCORE'],
             'dividends': contracts['dividends']['SCORE'],
             'oracle': contracts['dummy_oracle']['SCORE'],
             'sicx': contracts['sicx']['SCORE'],
             'icd': contracts['icd']['SCORE'],
             'bal': contracts['bal']['SCORE'],
             'bwt': contracts['bwt']['SCORE']
            }

class DistPercentDict(TypedDict):
    recipient_name : str
    bal_token_dist_percent: int

_recipient_list: List[DistPercentDict] = [{'recipient_name': 'Loans', 'bal_token_dist_percent': 25 * 10**16},
                   {'recipient_name': 'SICXICX', 'bal_token_dist_percent': 10 * 10**16},
                   {'recipient_name': 'SICXICD', 'bal_token_dist_percent': 20 * 10**16},
                   {'recipient_name': 'BALNICD', 'bal_token_dist_percent': 20 * 10**16},
                   {'recipient_name': 'Worker Tokens', 'bal_token_dist_percent': 20 * 10**16},
                   {'recipient_name': 'Reserve Fund', 'bal_token_dist_percent': 5 * 10**16}
                  ]

settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'sicx', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'staking', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'icd', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'bal', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'bwt', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'dex', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'reserve', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'rewards', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'dividends', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'rewards', 'method': 'addNewDataSource', 'params':{'_data_source_name': 'Loans', '_contract_address': contracts['loans']['SCORE']}},
            {'contract': 'rewards', 'method': 'addNewDataSource', 'params':{'_data_source_name': 'SICXICX', '_contract_address': contracts['dex']['SCORE']}},
            {'contract': 'rewards', 'method': 'addNewDataSource', 'params':{'_data_source_name': 'SICXICD', '_contract_address': contracts['dex']['SCORE']}},
            {'contract': 'rewards', 'method': 'addNewDataSource', 'params':{'_data_source_name': 'BALNICD', '_contract_address': contracts['dex']['SCORE']}},
            {'contract': 'rewards', 'method': 'updateBalTokenDistPercentage', 'params':{'_recipient_list': _recipient_list}},
            {'contract': 'loans', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['sicx']['SCORE'], '_active': 1, '_collateral': 1}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['icd']['SCORE'], '_active': 1, '_collateral': 0}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['bal']['SCORE'], '_active': 0, '_collateral': 1}},
            {'contract': 'governance', 'method': 'setAddresses', 'params':{'_addresses': addresses}},
            {'contract': 'governance', 'method': 'launchBalanced', 'params':{}}]


In [2039]:
# Cell 13

results = {}
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    results[f'{sett["contract"]}|{sett["method"]}|{sett["params"]}'] = res
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')



Calling setAdmin, with parameters {'_admin': 'cx21ecd939487863d4528d735543342d8e87cbfe01'} on the sicx contract.
Retrying in 1 seconds...
Status: 1


Calling setStakingAddress, with parameters {'_address': 'cx21ecd939487863d4528d735543342d8e87cbfe01'} on the sicx contract.
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1


Calling setSicxAddress, with parameters {'_address': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19'} on the staking contract.
Retrying in 1 seconds...
Status: 1


Calling setGovernance, with parameters {'_address': 'cx6eb8cc8c2ddc12021e8cb48d30aeb6afa8524aba'} on the icd contract.
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1


Calling setGovernance, with parameters {'_address': 'cx6eb8cc8c2ddc12021e8cb48d30aeb6afa8524aba'} on the bal contract.
Retrying in 1 seconds...
Status: 1


Calling setGovernance, with parameters {'_address': 'cx6eb8cc8c2ddc12021e8cb48d30aeb6afa8524aba'} on the bwt contract.
Retrying in 1 seconds...
Retrying in 2 second

## Run through read-only methods on all SCOREs

In [2009]:
# Cell 14

ro_methods = {"loans": ["name", "getAssetTokens", "getCollateralTokens", "getTotalCollateral", "getAvailableAssets", "assetCount", "getSnapshot"],
              "staking": ["name", "getSicxAddress", "getTodayRate"],
              "dividends": ["name", "getBalances"],
              "reserve": ["name", "getBalances"],
              "rewards": ["name", "getDataSourceNames"],
              "governance": ["name", "getAddresses"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin", "lastPriceInLoop"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "bal": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [2013]:
# Cell 15
# Runs through the readonly methods defined above. These can also be viewed using the testing UI

results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    print(f'Address: {contracts[score]["SCORE"]}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        if method in ['totalSupply', 'decimals', 'get_total_collateral', 'get_rate']:
            result = int(result, 0)
            if method in ['totalSupply', 'get_total_collateral', 'get_rate']:
                result = result / ICX
        print(f'{method}: {result}')
        results[score][method] = result


SCORE: loans
Address: cxd137a26fd8a731973592e793ee40d6a4a153dfdf
name: BalancedLoans
getAssetTokens: {'sICX': 'cx78be1157f950ddc2338bcac299d22023a887f7f5', 'ICD': 'cx150580e08f1f2244ca8cce47ed2b069b020ab03d', 'BALN': 'cxe3a6fec96db4ca31276827403b729c3ba311cb67'}
getCollateralTokens: {'sICX': 'cx78be1157f950ddc2338bcac299d22023a887f7f5', 'BALN': 'cxe3a6fec96db4ca31276827403b729c3ba311cb67'}
getTotalCollateral: 0x2b5e3af16b18800000
getAvailableAssets: {'sICX': {'symbol': 'sICX', 'address': 'cx78be1157f950ddc2338bcac299d22023a887f7f5', 'peg': 'sICX', 'added': '0x5bb5eb84a2626', 'is_collateral': '0x1', 'active': '0x1', 'bad_debt': '0x0', 'liquidation_pool': '0x0', 'dead_market': '0x0'}, 'ICD': {'symbol': 'ICD', 'address': 'cx150580e08f1f2244ca8cce47ed2b069b020ab03d', 'peg': 'USD', 'added': '0x5bb5eb868b61d', 'is_collateral': '0x0', 'active': '0x1', 'bad_debt': '0x0', 'liquidation_pool': '0x0', 'dead_market': '0x0'}}
assetCount: 0x3
getSnapshot: {'snap_day': '0x1', 'snap_time': '0x0', 'tot

### Get sICX from Staking

In [2040]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params({})\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x2b5e3af16b18800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x2b5e3af16b18800000', '0x4e6f6e65']} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x2b5e3af16b18800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx21ecd939487863d4528d735543342d8e87cbfe01', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x2b5e3af16b18800000'], 'data': ['800.0 sICX minted to hxe7af5fcfd8dfc67530a01a0e403882687528dfc

## Deposit ICX and Mint ICD Loan

In [2041]:
# Cell 16

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"ICD\", \"_amount\": 40000000000000000000}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'cx21ecd939487863d4528d735543342d8e87cbfe01', '0x2b5e3af16b18800000'], 'data': []} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x2b5e3af16b18800000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687865376166356663666438646663363735333061303161306534303338383236383735323864666362222c20225f6173736574223a2022494344222c20225f616d6f756e74223a2034303030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x2b5e3af16

### Just deposit collateral

In [2042]:
# Cell 17

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(30 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'cx21ecd939487863d4528d735543342d8e87cbfe01', '0x1a055690d9db80000'], 'data': []} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x1a055690d9db80000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687865376166356663666438646663363735333061303161306534303338383236383735323864666362222c20225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x1a055690d9db80000', '0x7b226d6

### Repay Some ICD

In [2043]:
# Cell 18

data = "{\"method\": \"_repay_loan\", \"params\": {}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', '0x4563918244f40000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'indexed': ['Burn(Address,int)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407'], 'data': ['0x4563918244f40000']} 

{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['LoanRepaid(Address,str,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'ICD', '0x4563918244f40000'], 'data': ['Loan of 5.0 ICD repaid to Balanced.']} 

{'scoreAddress': 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', '0x4563918244f40000'], 'data': ['0x7b226d6574686f64223a20225f7265

### Withdraw Some Collateral

In [2044]:
# Cell 19

params = {'_value': 25 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdrawCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x15af1d78b58c40000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x15af1d78b58c40000'], 'data': ['Collateral withdrawn. 25000000000000000000 sICX sent to hxe7af5fcfd8dfc67530a01a0e403882687528dfcb.']} 



### Add collateral directly

In [2045]:
data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 20 * ICX, '_data': data1 + b'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb' + data2}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['CollateralReceived(Address,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'sICX'], 'data': []} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', '0x1158e460913d00000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687865376166356663666438646663363735333061303161306534303338383236383735323864666362222c20225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']} 



### Test Liquidation

In [2046]:
# Cell 20
# 1. Deposit collateral to fresh wallet

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(782769 * ICX // 1000)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'cx21ecd939487863d4528d735543342d8e87cbfe01', '0x2a6f1a22364bbe8000'], 'data': []} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x2a6f1a22364bbe8000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687833663031383430613539396461303762306636323065656165376161396335373431363961346265222c20225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Mint(Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x2a6f1a22364bbe8000', '0x7b226d6574686f64223a20225f646

In [2047]:
# Cell 21
# 2. Check the account standing after depositing collateral - Should be 'No Debt', and
#    holding '0x2a6f1a22364bbe8000' worth of sICX.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'pos_id': '0x2',
 'created': '0x5bb5fa5b0eda9',
 'updated': '0x5bb5fa5b0eda9',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'snap_id': '0x1',
 'snaps_length': '0x1',
 'last_snap': '0x1',
 'assets': {'sICX': '0x2a6f1a22364bbe8000'},
 'replay_index': '0x0',
 'total_debt': '0x0',
 'ratio': '0x0',
 'standing': 'No Debt'}

#### 3. Update the loans SCORE to add a loan of ICD to the Balanced test wallet
Code in the on_update() method will mint and add the ICD. Adds an undercollateralized amount of ICD, 200 ICD.

In [2050]:
# Cell 22
# 3a. May need to run this a few times to get the position below liquidate threshold.

contract = 'loans'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

# step_limit = icon_service.estimate_step(deploy_transaction) + 100000
step_limit = 3000000000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0xad78ebc5ac6200000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'indexed': ['Mint(Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0xad78ebc5ac6200000', '0x4e6f6e65']} 

{'scoreAddress': 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx0478fad07631c13f58b5527e05397845cb3fb1e9'], 'data': ['0x2454d8dcb2cf44bb']} 



In [2051]:
# Cell 23
# 4. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' ICD and have standing of 'Liquidate'.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'pos_id': '0x2',
 'created': '0x5bb5fa5b0eda9',
 'updated': '0x5bb5facda406e',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'snap_id': '0x1',
 'snaps_length': '0x1',
 'last_snap': '0x1',
 'assets': {'sICX': '0x2a6f1a22364bbe8000', 'ICD': '0xad78ebc5ac6200000'},
 'replay_index': '0x0',
 'total_debt': '0x1c62496c6bb1edb218',
 'ratio': '0x14bf4efe169d20b0',
 'standing': 'Liquidate'}

In [2052]:
# Cell 24
# 5. Replay events to update standing for this position.

params = {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'} # wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(100000000)\
    .nid(NID)\
    .nonce(100)\
    .method("updateStanding")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1


In [1846]:
347200 * 12500000000 / 10**18

0.00434

In [2053]:
res

{'txHash': '0x36b35b76a126250a83ac673ac0d2603391cbca4b628c712a8ed4ab304a6e814a',
 'blockHeight': 165787,
 'blockHash': '0x42dff506ebc03aff7517dfc6b1eebb6cd62579d9cb01eabc8c6aeffb287428ce',
 'txIndex': 1,
 'to': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407',
 'stepUsed': 245480,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 245480,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [2054]:
params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('hasDebt')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

'0x1'

In [2055]:
# Cell 25
# 6. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' ICD and have standing of 'Liquidate'.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'pos_id': '0x2',
 'created': '0x5bb5fa5b0eda9',
 'updated': '0x5bb5fad35e18a',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'snap_id': '0x1',
 'snaps_length': '0x1',
 'last_snap': '0x1',
 'assets': {'sICX': '0x2a6f1a22364bbe8000', 'ICD': '0xad78ebc5ac6200000'},
 'replay_index': '0x0',
 'total_debt': '0x1c62496c6bb1edb218',
 'ratio': '0x14bf4efe169d20b0',
 'standing': 'Liquidate'}

In [2056]:
# Cell 24
# 5. Liquidate the account position.

params = {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("liquidate")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x48c86487af753800'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x48c86487af753800'], 'data': ['Liquidation reward of 5244552300000000000 sICX sent to hxe7af5fcfd8dfc67530a01a0e403882687528dfcb.']} 

{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['Liquidate(Address,int,str)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x2a6f1a22364bbe8000'], 'data': ['782769000000000000000 liquidated from hx3f01840a599da07b0f620eeae7aa9c574169a4be']} 



In [2057]:
# Cell 25
# 6. Check the standing of the account after liquidation. Should now have zero balance for sICX and ICD.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'pos_id': '0x2',
 'created': '0x5bb5fa5b0eda9',
 'updated': '0x5bb5fae85ef8a',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'snap_id': '0x1',
 'snaps_length': '0x1',
 'last_snap': '0x1',
 'assets': {'sICX': '0x0', 'ICD': '0x0'},
 'replay_index': '0x0',
 'total_debt': '0x0',
 'ratio': '0x14bf4efe169d20b0',
 'standing': 'No Debt'}

In [ ]:
# Create a set of wallets

# wallets = []
# for i in range(200):
#     wallets.append(KeyWallet.create())
# with open("../wallets.pkl", 'wb') as f:
#     pkl.dump(wallets, f)

In [2058]:
with open("../wallets.pkl", 'rb') as f:
    wallets = pkl.load(f)

In [2060]:
# 1000 ICX into each wallet. These wallets are already more than well stocked enough on the tbears server.

for i in range(200):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(wallets[i].get_address())\
        .value(1000 * ICX)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0xab5fda9a7326d5bff0e8ba8961ca8429fdc851d62e270d9a51a9912b705073b7'

In [2061]:
# Mint some ICD to each wallet

for i in range(0, 10):
    data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
    data2 = "\", \"_asset\": \"ICD\", \"_amount\": 4000000000000000000}}".encode("utf-8")
    params = {'_data1': data1, '_data2': data2}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(80 * ICX)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("addCollateral")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Status: 1
{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'cx21ecd939487863d4528d735543342d8e87cbfe01', '0x4563918244f400000'], 'data': []} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'hx4c554a3373a41f71eae5413e9af4c5faab590265', '0x4563918244f400000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687834633535346133333733613431663731656165353431336539616634633566616162353930323635222c20225f6173736574223a2022494344222c20225f616d6f756e74223a20343030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Mint(Address,int,bytes)', 'hx4c554a3373a41f71eae5413e9af4c5faab590265'], 'data': ['0x4563918244f400000', '0x7b226d6574686f6

In [2062]:
res

{'txHash': '0x3c15a163f76d849f31dd5ea2b44fad81fcedd211907960840ecaa101c7fadf87',
 'blockHeight': 166010,
 'blockHash': '0xf04e376d6aca53042a54e94b3693d273c71fddc35ebf19710890a3c7fb00f852',
 'txIndex': 1,
 'to': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407',
 'stepUsed': 896400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 896400,
 'eventLogs': [{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407',
    'cx21ecd939487863d4528d735543342d8e87cbfe01',
    '0x4563918244f400000'],
   'data': []},
  {'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19',
    'hx4c554a3373a41f71eae5413e9af4c5faab590265',
    '0x4563918244f400000'],
   'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687834633535346

In [2063]:
i

9

In [2064]:
wallets[2].get_address()

'hxc6077c30d47d9a03f1ad5fcfddb684a1b3fc9d89'

In [2065]:
data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"ICD\", \"_amount\": 4000000000000000000}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallets[2].get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(80 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallets[2])
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Status: 1
{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'cx21ecd939487863d4528d735543342d8e87cbfe01', '0x4563918244f400000'], 'data': []} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'hxc6077c30d47d9a03f1ad5fcfddb684a1b3fc9d89', '0x4563918244f400000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687863363037376333306434376439613033663161643566636664646236383461316233666339643839222c20225f6173736574223a2022494344222c20225f616d6f756e74223a20343030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Mint(Address,int,bytes)', 'hxc6077c30d47d9a03f1ad5fcfddb684a1b3fc9d89'], 'data': ['0x4563918244f400000', '0x7b226d6574686f6

In [2066]:
# Mint some ICD to each wallet

for i in range(0, 100):
    data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
    data2 = "\", \"_asset\": \"ICD\", \"_amount\": 40000000000000000000}}".encode("utf-8")
    params = {'_data1': data1, '_data2': data2}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(800 * ICX)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("addCollateral")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Status: 1
{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'cx21ecd939487863d4528d735543342d8e87cbfe01', '0x2b5e3af16b18800000'], 'data': []} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'hx06fe1b158d82669a578d77b29f699b959f287fe2', '0x2b5e3af16b18800000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687830366665316231353864383236363961353738643737623239663639396239353966323837666532222c20225f6173736574223a2022494344222c20225f616d6f756e74223a2034303030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Mint(Address,int,bytes)', 'hx06fe1b158d82669a578d77b29f699b959f287fe2'], 'data': ['0x2b5e3af16b18800000', '0x7b226d6574

In [2067]:
# redeem 20 ICD from each of 30 wallets that do not have positions on Balanced. This will use up all
# of the liquidation pool and require some replay events to be recorded.

for i in range(30, 90):
    params = {'_to': wallets[i+100].get_address(), '_value': 20 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['icd']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)
    data = "{\"method\": \"_retire_asset\", \"params\": {}}".encode("utf-8")
    params = {'_to': contracts['loans']['SCORE'], '_value': 10 * ICX, '_data': data}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i+100].get_address())\
        .to(contracts['icd']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i+100])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', '0x8ac7230489e80000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx249d6262fdc03fdc791a556ef92d8560f9818b77', 'indexed': ['Burn(Address,int)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407'], 'data': ['0x8ac7230489e80000']} 

{'scoreAddress': 'cxcfc39861330419a80f0d9f37a4cc8b7d80605a19', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'hx9b4df089e809af4c2466570b05140b9880f3e3b8', '0x16b50789efc18af4e'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx307204973ee008ddbbd17ac0a9d7e7e36f4d5407', 'indexed': ['TokenTransfer(Address,int,str)', 'hx9b4df089e809af4c2466570b05140b9880f3e3b8', '0x16b50789efc18af4e'], 'data': ['Collateral redeemed. 26179557258134335310 sICX sent to hx9b4df089e809af4c2466570b05140b9

In [2070]:
for i in range(0, 100):
    params = {'_owner': wallets[i].get_address()}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

0
Retrying in 1 seconds...
Retrying in 2 seconds...
Retrying in 4 seconds...
Retrying in 8 seconds...
Retrying in 16 seconds...
Retrying in 32 seconds...


KeyboardInterrupt: 

In [2071]:
# Check what has been distributed on the rewards SCORE.

params = {'_holders': [wallet.get_address() for wallet in wallets[:110]]}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
i = 0
{i: [key, result[key]] for i, key in enumerate(result.keys())}

ReadTimeout: HTTPConnectionPool(host='52.53.175.151', port=9000): Read timed out. (read timeout=10)

In [ ]:
params = {}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['rewards']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("claimRewards")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

print(i)
res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(0, 100):
    params = {}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['rewards']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("claimRewards")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

In [ ]:
params = {'_name': 'Loans', '_snapshot_id': 0, '_limit': 10, '_offset': 0}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method("getDataBatch")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
params = {'_owner': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("updateStanding")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [2072]:
icon_service.get_balance(wallets[66].get_address()) / 10**18

ReadTimeout: HTTPConnectionPool(host='52.53.175.151', port=9000): Read timed out. (read timeout=10)

In [ ]:
for i in range(30):
    print(wallets[i].get_address())

In [ ]:
for i in range(100, 130):
    print(wallets[i].get_address())

In [ ]:
addresses = [wallet.get_address() for wallet in wallets]

In [ ]:
balances = {}
for address in addresses[:20]:
    params = {'_owner': address}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['bal']['SCORE'])\
                        .method('availableBalanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

In [ ]:
total_baln = 0
for address in balances:
    total_baln += int(balances[address], 0)

In [ ]:
for contract in contracts:
    addresses.append(contracts[contract]['SCORE'])

In [ ]:
balances

### DEX SICX Test

Does the following:

1. Sends ICX to the DEX
2. Checks overall ICX balance
3. Checks withdrawal lock
4. Mints some SICX
5. Trades SICX against ICX

In [ ]:
# Cell 26

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 27

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
datetime.utcfromtimestamp(1613209960533864 // 10**6).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# Cell 28

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXWithdrawLock')\
                    .build()
result = icon_service.call(call)
datetime.utcfromtimestamp(int(result, 0) // 10**6).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# Cell 29

params = {'_to': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 30

data = "{\"method\": \"_swap_icx\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(30000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 31

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 32

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 33

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDataBatch')\
                    .params({'_name': 'SICXICX', '_limit': 10, '_snapshot_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 34

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 35

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 36

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bal']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 37

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDeposit')\
                    .params({'_user': wallet.get_address(), '_tokenAddress': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 38

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['icd']['SCORE'], '_baseValue': 1 * ICX, '_quoteValue': 1 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 39

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("permit")\
    .params({'_pid': 1, '_permission': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 40

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getInversePrice')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 41

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 42

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolId')\
                    .params({'_token1Address': contracts['sicx']['SCORE'], '_token2Address': contracts['icd']['SCORE'] }) \
                    .build()
result = icon_service.call(call)
print("pool w/ tokens: " + str(contracts['sicx']['SCORE']) + ' & ' + str(contracts['icd']['SCORE']))
print(result)

In [ ]:
# Cell 43

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolBase')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolQuote')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':2, '_name': 'SICXICD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 45

to_token = contracts['sicx']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":10}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX / 100), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 45

to_token = contracts['icd']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX / 100), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 46

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['sicx']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 47

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 48

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('totalSupply')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 49

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("remove")\
    .params({'_pid':2, '_value': 1 * ICX, '_withdraw': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 50

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 30 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 51

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 52

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("cancelSicxicxOrder")\
    .params({}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 38

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['bal']['SCORE'], '_quoteToken': contracts['icd']['SCORE'], '_baseValue': 100 * ICX, '_quoteValue': 10 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':3, '_name': 'BALNICD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
to_token = contracts['bal']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')